In [ ]:
G_path = '/.../G_net.pkl'
Ds_path = '/.../D_seq_net.pkl'
Di_path = '/.../D_img_net.pkl'

In [ ]:
G.save(G_path)
D_img.save(Di_path)
D_seq.save(Ds_path)

In [ ]:
root = '/.../'
step = 0
G_LOADPATH = '/.../G_net.pkl'
with torch.no_grad():
    for x,y in zip(test_dataloader,my_imgG1loader):
        waveform,sample_rate = x
        image = y
        z = Variable(torch.randn(waveform.shape[0],10)).to(device)
        waveform=waveform.to(device)
        image=image.to(device)
        G.load(G_LOADPATH)
        fake = G(image,waveform).cpu()
        print(fake.shape)
        plt.figure(figsize=(200,200))
        x,y =1,1
        to_image = Image.new('RGB', (200 * 4, 4 * 200))
        for i in range(1,BATCH_SIZE):
            step += 1
            fake1= fake[i]
            input = fake1.mul_(0.5).add_(0.5).mul_(255).clamp_(0, 255).permute(1, 2, 0).type(torch.uint8).numpy()
            input_tensor = cv2.cvtColor(input, cv2.COLOR_RGB2BGR)
            img_path=root+"Image"+str(step)+".jpg"
            
            cv2.imwrite(img_path, input_tensor)
            im = Image.open(img_path)
            to_image.paste(im, ((x - 1) * 200, (y - 1) * 200))
            if x>4:
                x =1 
                y+=1
            else : x+=1
            plt.subplot(1,17,i)
            plt.imshow(np.transpose(vutils.make_grid(fake[i],padding=0,normalize=True),(1,2,0)))
            plt.xticks([])
            plt.yticks([])
        plt.show()
        to_image.save("/..../"+str(step)+".jpg","JPEG")

In [ ]:
def picvideo():
    path ='/.../'
    filelist = os.listdir(path) 
    
    filelist.sort(key=lambda x:int(x[5:-4]))
    fps = 8
    size = (200,200) 
    file_path = "/.../" + str(int(time.time())) + ".mp4"
    fourcc = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X')
    video = cv2.VideoWriter( file_path, fourcc, fps, size )
    for item in filelist:
        if item.endswith('.jpg'):   
            item = path + '/' + item 
            img = cv2.imread(item) 
            res=cv2.resize(img,(200,200),interpolation=cv2.INTER_CUBIC) 
            video.write(res)      
    video.release() 

In [ ]:
def test_pretreatment_wav(main_wav_path, part_wav_path):
    for i in range(60000):
        if i%120 == 0 :
            start_time = int(i-120)
            if start_time < 0:
                start_time = 0.
            end_time = int(i+120)

            sound = AudioSegment.from_mp3(main_wav_path)
            word = sound[start_time:end_time]

            word.export(part_wav_path+str(i)+'.wav', format="wav")